In [2]:
from lets_plot import *
import numpy as np
import pandas as pd
import os
from system_visualize import data2df, cluster_single_df
import polars as pl
LetsPlot.setup_html()

In [3]:
df_all = pl.DataFrame()
for case in [100,280,300,350,400]:
    coor = data2df(f'./data/data{case}.extra')
    df_cluster = cluster_single_df(coor)
    df = pl.from_pandas(df_cluster)
    df = df.with_columns(pl.col("clusterID").fill_null(0).cast(pl.Int32),
                pl.col(['type',"atomID"]).cast(pl.Int32))
    print(case)
    df = df.filter(pl.col('type') == 5, pl.col('clusterID')>0) 
    counts = df.group_by('clusterID').agg(pl.col('clusterID').count().alias('size')).sort('size').with_columns(pl.lit(f'{case/100}').alias('kT'))
    counts = counts.with_columns(size_std = pl.col('size').std(),
                                 size_mean = pl.col('size').mean(),
                                 size_upper = (pl.col('size').mean()+pl.col('size').std()),
                                 size_lower = (pl.col('size').mean()-pl.col('size').std()))
    df_all = pl.concat([df_all, counts])
df_all.sample(12)

100
280
300
350
400


clusterID,size,kT,size_std,size_mean,size_upper,size_lower
i32,u32,str,f64,f64,f64,f64
17,20,"""2.8""",5.174883,22.176471,27.351353,17.001588
13,14,"""3.0""",8.285796,26.823529,35.109325,18.537734
12,26,"""3.0""",8.285796,26.823529,35.109325,18.537734
2,24,"""2.8""",5.174883,22.176471,27.351353,17.001588
10,26,"""3.0""",8.285796,26.823529,35.109325,18.537734
…,…,…,…,…,…,…
12,19,"""2.8""",5.174883,22.176471,27.351353,17.001588
9,17,"""2.8""",5.174883,22.176471,27.351353,17.001588
4,24,"""2.8""",5.174883,22.176471,27.351353,17.001588


In [9]:
(
    ggplot(df_all, aes(x='kT', y='size_mean', color='kT')) +
    geom_point(size=12)+
    geom_line(aes(x='kT',y='size_mean'),color='black')+
    geom_errorbar(aes(ymin='size_lower', ymax='size_upper'), width=0.3)+
    theme_classic()
)